In [2]:
!pip install pyspark
!pip install findspark

In [13]:
from pyspark.sql import SparkSession
import findspark

findspark.init()

spark = SparkSession.builder.appName("Pyspark ProjetoBD")\
                    .config("spark.memory.offHeap.enabled","true")\
                    .config("spark.memory.offHeap.size","20g")\
                    .getOrCreate()
spark


In [22]:
df = spark.read.csv(r'C:\Users\user\Desktop\ESTB\3 ano\1º semestre\Big Data\Trabalho 2\bodyPerformance.csv',header=True,sep=";")

In [23]:
print("Numero de Linhas:", df.count())
print("Numero de Colunas:", len(df.columns))
print("Esquema do DataFrame: ")
df.printSchema()

Numero de Linhas: 13393
Numero de Colunas: 12
Esquema do DataFrame: 
root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- height_cm: string (nullable = true)
 |-- weight_kg: string (nullable = true)
 |-- body fat_%: string (nullable = true)
 |-- diastolic: string (nullable = true)
 |-- systolic: string (nullable = true)
 |-- gripForce: string (nullable = true)
 |-- sit and bend forward_cm: string (nullable = true)
 |-- sit-ups counts: string (nullable = true)
 |-- broad jump_cm: string (nullable = true)
 |-- class: string (nullable = true)



In [8]:
df.show(10)

+----+------+---------+---------+----------+---------+--------+---------+-----------------------+--------------+-------------+-----+
| age|gender|height_cm|weight_kg|body fat_%|diastolic|systolic|gripForce|sit and bend forward_cm|sit-ups counts|broad jump_cm|class|
+----+------+---------+---------+----------+---------+--------+---------+-----------------------+--------------+-------------+-----+
|27.0|     M|    172.3|    75.24|      21.3|     80.0|   130.0|     54.9|                   18.4|          60.0|        217.0|    C|
|25.0|     M|    165.0|     55.8|      15.7|     77.0|   126.0|     36.4|                   16.3|          53.0|        229.0|    A|
|31.0|     M|    179.6|     78.0|      20.1|     92.0|   152.0|     44.8|                   12.0|          49.0|        181.0|    C|
|32.0|     M|    174.5|     71.1|      18.4|     76.0|   147.0|     41.4|                   15.2|          53.0|        219.0|    B|
|28.0|     M|    173.8|     67.7|      17.1|     70.0|   127.0|     4